# Optimizar espacio para trabajar con DataFrames

Cuando importamos datos de un fichero y los cargamos en un DataFrame, debemos de estar atentos a cómo se guarda la información ya que podría estar usándose más espacio en memoria del necesario.

In [1]:
import pandas as pd
df = pd.read_csv("datosAImportar.csv")

In [2]:
#Exploramos los datos cargados
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
Dia           100000 non-null object
Mes           100000 non-null object
Cantidad      100000 non-null float64
Candidatos    100000 non-null int64
Tiempo        100000 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 3.8+ MB


Haciendo uso del atributo memory_usage="deep", podemos obtener una información más precisa sobre el espacio que ocupa un DataFrame.

In [3]:
df.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
Dia           100000 non-null object
Mes           100000 non-null object
Cantidad      100000 non-null float64
Candidatos    100000 non-null int64
Tiempo        100000 non-null object
dtypes: float64(1), int64(1), object(3)
memory usage: 20.3 MB


In [4]:
df.head()

,Dia,Mes,Cantidad,Candidatos,Tiempo
0,Domingo,Junio,437.195611,5,lluvia
1,Lunes,Febrero,366.766836,10,lluvia
2,Martes,Noviembre,406.538866,12,nieve
3,Jueves,Marzo,658.514502,9,lluvia
4,Miércoles,Abril,592.096966,6,nieve


De cara a optimizar el espacio en memoria, vamos a ver cuánto está ocupando, en media, una columna dependiendo del tipo que almacena.

In [5]:
def espacioPorTipo(df):
    for tipo in ['float','int','object']:
        columnasTipo = df.select_dtypes(include=[tipo])
        mediaUsoMemoria = columnasTipo.memory_usage(deep=True).mean()
        mediaUsoMemoriaMB = mediaUsoMemoria / 1024 ** 2
        print("Uso de memoria para el tipo ",tipo , " : {:0.5f} MB".format(mediaUsoMemoriaMB))

In [6]:
espacioPorTipo(df)

Uso de memoria para el tipo  float  : 0.38151 MB
Uso de memoria para el tipo  int  : 0.38151 MB
Uso de memoria para el tipo  object  : 4.69722 MB


Vemos que las columnas de tipo object son las que más ocupan con diferencia. 

Vamos a ver cómo optimizar el uso de memoria de cada tipo paso a paso.

## Optimizando el espacio que ocupan las columnas de números

Tenemos una sola columna de tipo entero, donde podemos observar que se almacenan los números usando 64 bits. Con esta cantidad de bits, podemos representar números muy grandes que quizás no estemos utilizando. Vamos a ver cómo son los datos que almacenamos

In [7]:
df["Candidatos"].describe()

count    100000.000000
mean         10.474550
std           5.750018
min           1.000000
25%           6.000000
50%          10.000000
75%          15.000000
max          20.000000
Name: Candidatos, dtype: float64

Observamos que tenemos números entre 1 y 20. Por tanto, podemos representarlos usando menos información. Para ello, usamos el método to_numeric de pandas y el atributo downcast="unsigned" para especificar números sin signo (ya que todos son positivos). Convertimos las columnas que contienen enteros y vemos el resultado.

El método to_numeric con el atributo downcast permite almacenar un número en el tipo más pequeño posible.

Si tuviésemos números positivos y negativos, usaríamos la opción integer.

In [8]:
#reducimos espacio de las columnas de tipo entero
df2 = df.copy()
for col in df2.columns:            
       if "int" in str(df2[col].dtype):
            df2[col] = pd.to_numeric(df2[col],downcast='unsigned')

In [9]:
df2.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
Dia           100000 non-null object
Mes           100000 non-null object
Cantidad      100000 non-null float64
Candidatos    100000 non-null uint8
Tiempo        100000 non-null object
dtypes: float64(1), object(3), uint8(1)
memory usage: 19.6 MB


In [10]:
espacioPorTipo(df2)

Uso de memoria para el tipo  float  : 0.38151 MB
Uso de memoria para el tipo  int  : 0.00008 MB
Uso de memoria para el tipo  object  : 4.69722 MB


Realizamos el mismo proceso con los números en coma flotante usando el valor float para el atributo downcast

In [11]:
#reducimos espacio de las columnas de tipo float
df3 = df2.copy()
for col in df3.columns:            
       if "float" in str(df3[col].dtype):
            df3[col] = pd.to_numeric(df3[col],downcast='float')

In [12]:
df3.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
Dia           100000 non-null object
Mes           100000 non-null object
Cantidad      100000 non-null float32
Candidatos    100000 non-null uint8
Tiempo        100000 non-null object
dtypes: float32(1), object(3), uint8(1)
memory usage: 19.3 MB


In [13]:
espacioPorTipo(df3)

Uso de memoria para el tipo  float  : 0.00008 MB
Uso de memoria para el tipo  int  : 0.00008 MB
Uso de memoria para el tipo  object  : 4.69722 MB


## Reducir espacio del tipo object

Tenemos que explorar el tipo para ver cómo podemos reducirlo. Si nos fijamos, está usándose texto para representar una serie de categorías. De hecho, a través de unique podemos ver cuántas categorías distintas tenemos

In [14]:
df3["Mes"].unique()

array(['Junio', 'Febrero', 'Noviembre', 'Marzo', 'Abril', 'Mayo',
       'Diciembre', 'Enero', 'Octubre', 'Agosto', 'Julio', 'Septiembre'],
      dtype=object)

In [15]:
df3["Dia"].unique()

array(['Domingo', 'Lunes', 'Martes', 'Jueves', 'Miércoles', 'Viernes',
       'Sábado'], dtype=object)

In [16]:
df3["Tiempo"].unique()

array(['lluvia', 'nieve', 'nublado', 'soleado'], dtype=object)

En los tres casos tenemos como mucho 12 valores distintos por categoría. Sería más eficiente guardar la información asociada a la categoría en lugar del texto, que ocupa mucho más. Usando categorías, una columna puede ocupar el mismo espacio que si usásemos enteros. 

Probemos a convertir las columnas de tipo objeto a categoría.

In [17]:
df4 = df3.copy()
for col in df4.columns:            
       if "object" in str(df4[col].dtype):
            df4[col] = df4[col].astype('category')

In [18]:
df4.info(memory_usage="deep")

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 5 columns):
Dia           100000 non-null category
Mes           100000 non-null category
Cantidad      100000 non-null float32
Candidatos    100000 non-null uint8
Tiempo        100000 non-null category
dtypes: category(3), float32(1), uint8(1)
memory usage: 783.6 KB


In [19]:
espacioPorTipo(df4)

Uso de memoria para el tipo  float  : 0.00008 MB
Uso de memoria para el tipo  int  : 0.00008 MB
Uso de memoria para el tipo  object  : 0.00008 MB


In [20]:
df4.head()

,Dia,Mes,Cantidad,Candidatos,Tiempo
0,Domingo,Junio,437.195618,5,lluvia
1,Lunes,Febrero,366.766846,10,lluvia
2,Martes,Noviembre,406.538879,12,nieve
3,Jueves,Marzo,658.514526,9,lluvia
4,Miércoles,Abril,592.096985,6,nieve


# Pregunta abierta para experimentar

¿Qué pasaría si importamos datos que contienen fechas? 

In [21]:
#trabajar con la siguiente colección que contiene fechas
dfConFechas = pd.read_csv("datosAImportarConFechas.csv")